RAIN - Real & Artificial Intelligence for Neuroscience

## Automatic analysis
- Use a trained model from [3a-Create_models](3a-Create_models.ipynb) to label the position files.

#### Requirements:
- The position.csv files.
- A trained model.

---
#### Load the necessary modules

In [ ]:
from pathlib import Path
import rainstorm.modeling as rst

---
#### 1. State your project path
You need to define the path to the same folder used in [2a-Prepare_positions](2a-Prepare_positions.ipynb), and the path to the parameters file (which contains the parameters for automatic analysis).

On the params.yaml file, **change the model_path for the path to the trained model** created in [3a-Create_models](3a-Create_models.ipynb).

In [ ]:
base = Path.cwd()
folder_path = base / 'examples' / 'NOR'
params = folder_path / 'params.yaml'

---
#### Open the params.yaml file and make sure the parameters match the model you want to use for analysis:

In [ ]:
rst.open_params_editor(params)

Automatic Analysis

- `models_path` : Path to the models folder containing trained neural networks.
- `analyze_with` : **Model file to use for analysis** (.keras format).
- `model_bodyparts` : Bodyparts used to train the model.

RNN Configuration

- `rescaling` : Whether to rescale the data  
- `reshaping` : Whether to reshape the data (set to True for RNN)  
- `RNN_width` : Defines the shape of the RNN  
  - `past` : Number of past frames to include  
  - `future` : Number of future frames to include  
  - `broad` : Broaden the window by skipping some frames as we stray further from the present  

---
#### 2. Run the model on all the data selected

In [ ]:
rst.create_autolabels(params)

---
---
#### A new csv file was created for each video, containing the autolabels.

We can:
- Continue on this notebook and compare the results of the manual, geometric & automatic methods on our example files
- Skip the methods comparison, go straight to [4-Seize_labels](4-Seize_labels.ipynb) and use the autolabels to plot the results of the experiment
---

## Compare labels
Lets do a quick recap of what we have done so far...

- Recorded mice exploring objects.
- Aligned the videos using the Rainstorm Video Aligner, and...
- Drawn ROIs and targets present, and set a scaling factor using the drawing tool on [0-Video_handling](0-Video_handling.ipynb).
- Analyzed the videos using pose estimation software (like DeepLabCut).
- **Manually labeled** the positions of the mice using the Rainstorm Behavioral Tagger on [1-Behavioral_labeler](1-Behavioral_labeler.ipynb).
- Processed the positions on [2a-Prepare_positions](2a-Prepare_positions.ipynb).
- **Geometrically labeled** the positions on [2b-Geometric_analysis](2b-Geometric_analysis.ipynb).
- Trained some Artificial Neural Networks on [3a-Create_models](3a-Create_models.ipynb).
- **Automatically labeled** the positions on [3b-Automatic_analysis](3b-Automatic_analysis.ipynb).

And here we are! Now, we can compare between manual labels, geolabels & autolabels.

I have labeled the example videos myself using the RAINSTORM labeler tool, and left those manual labels available on the example folder.

If you want to see the original videos and label them to compare yourself with the machine, you can find the videos on 'examples/NOR/TS_videos'.

---
#### 3. Polar graph 
A great way of visualizing the distance and angle of approach to an object is to use a polar graph. The distance is represented in the radius of the circle, and the circumference represents the angle of the vector from the head to the nose.

Since the graph is symmetrical, we will use the left side to color the automatic labels in red, and the right side to color the manual labels in blue. The graph will also show the geometric labels as all the points that fit inside the dashed line.

In [ ]:
# We can open a single example file and it's labels
positions, manual_labels, geolabels, autolabels = rst.prepare_label_comparison(params, include_all=False)

rst.polar_graph(params, positions, autolabels, manual_labels, target_name='obj_2')

#### We can load all the data from all the available files and plot the polar graph again

In [ ]:
# This time we set 'include_all' to True
all_positions, all_manual_labels, all_geolabels, all_autolabels = rst.prepare_label_comparison(params, include_all=True)

# This time it might take a few seconds
rst.polar_graph(params, all_positions, all_autolabels, all_manual_labels, target_name='obj_2')

---
#### 4. Evaluate accuracy of predictions
Finally, we can evaluate the accuracy of the predictions by comparing the geometric and automatic labels to the manual labels.

In [ ]:
# For geolabels
rst.accuracy_scores(all_manual_labels, all_geolabels, targets=['obj_1','obj_2'], method_name="geometric")

In [ ]:
# For autolabels
rst.accuracy_scores(all_manual_labels, all_autolabels, targets=['obj_1','obj_2'], method_name="automatic")

---
---
#### Both geometric and automatic methods are flawed when compared to human labeling.
#### However, as the only human responsible for the example manual labels provided, let me tell you that I am flawed too.
#### Probably if more people labeled these videos and we compare against the average labels, we would have less error.

We can:
- Go to [4-Seize_labels](4-Seize_labels.ipynb) and use the labels to plot the results of the experiment.

---
RAINSTORM - Created on Apr 18, 2024 - @author: Santiago D'hers